https://cloud.google.com/ai-platform/training/docs/getting-started-scikit-xgboost

https://cloud.google.com/ai-platform/prediction/docs/using-pipelines-for-preprocessing#gcloud_2

In [8]:
# import sys
# modulePath = '/Users/kristine/WRI/MachineLearning/gfw-forestlearn'
# if modulePath not in sys.path:
#     sys.path.append(modulePath)
# from gfw_forestlearn import fl_binary_classification
# #from gfw_forestlearn import fl_regression
# #import pandas as pd
# import numpy as np
# from datetime import datetime
# import os
# import pandas as pd
# import subprocess
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math


In [9]:
dataDir = '/Users/kristine/Downloads/CongoGEE'
os.chdir(dataDir)
trainFile = 'train.csv'
testFile = 'test.csv'
validationFile = 'validation.csv'

train = pd.read_csv(trainFile)
validation = pd.read_csv(validationFile)


In [10]:
label = 'lossyear'
predictors = ['concessionsImage','conflictsDistance','earlyLossDistance','elevation', 'huntingAreasImage',
              'localitiesDistance', 'loggingDistance', 'lossyear', 'miningImage', 'navigableRiversDistance',
              'protectedAreasImage', 'roadsDistance', 'ruralComplexDistance', 'slope']

cat_feats = ['concessionsImage','miningImage','huntingAreasImage','protectedAreasImage']

all_cols = [label]+predictors

In [11]:
# Name of the label column.


print(list(train))
#print(train[label].unique())
trainFilt = train.drop(columns=[x for x in list(train) if x not in all_cols])
validationFilt = train.drop(columns=[x for x in list(validation) if x not in all_cols])

print(list(trainFilt))
#print(train[label].unique())

train_tf = tfdf.keras.pd_dataframe_to_tf_dataset(trainFilt, label=label)
validation_tf = tfdf.keras.pd_dataframe_to_tf_dataset(validationFilt, label=label)


['system:index', 'SplitValue', 'concessionsImage', 'conflictsDistance', 'earlyLossDistance', 'elevation', 'huntingAreasImage', 'localitiesDistance', 'loggingDistance', 'lossyear', 'miningImage', 'navigableRiversDistance', 'protectedAreasImage', 'random', 'roadsDistance', 'ruralComplexDistance', 'slope', '.geo']
['concessionsImage', 'conflictsDistance', 'earlyLossDistance', 'elevation', 'huntingAreasImage', 'localitiesDistance', 'loggingDistance', 'lossyear', 'miningImage', 'navigableRiversDistance', 'protectedAreasImage', 'roadsDistance', 'ruralComplexDistance', 'slope']


/usr/local/lib/python3.9/site-packages/tensorflow_decision_forests/keras/core.py:2036: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
2022-02-02 15:39:27.925062: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:

# Specify the model.
model_1 = tfdf.keras.RandomForestModel()#features=predictors,exclude_non_specified_features=True)
# Optionally, add evaluation metrics.
model_1.compile(metrics=["accuracy"]) #

# Train the model.
# "sys_pipes" is optional. It enables the display of the training logs.
model_1.fit(x=train_tf)

evaluation = model_1.evaluate(validation_tf, return_dict=True)

for name, value in evaluation.items():
    print(f"{name}: {value:.4f}")

model_1.save("model")


Use /var/folders/ls/m3fnvjqj1qs_hsp9k3s_q1h00000gp/T/tmp6c_r4hs5 as temporary training directory
Starting reading the dataset
1/5 [=====>........................] - ETA: 14s
Dataset read in 0:00:03.759500
Training model
Model trained in 0:00:00.483239
Compiling model
5/5 [==============================] - 4s 169ms/step


[INFO kernel.cc:1153] Loading model from path
[INFO decision_forest.cc:617] Model loaded with 300 root(s), 59776 node(s), and 13 input feature(s).
[INFO abstract_model.cc:1063] Engine "RandomForestOptPred" built
[INFO kernel.cc:1001] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
5/5 [==============================] - 0s 16ms/step - loss: 0.0000e+00 - accuracy: 0.9808
loss: 0.0000
accuracy: 0.9808


2022-02-02 15:39:35.050432: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [ ]:
# from tensorflow.python.tools import saved_model_utils

# meta_graph_def = saved_model_utils.get_meta_graph_def('model', 'serve')
# inputs = meta_graph_def.signature_def['serving_default'].inputs
# outputs = meta_graph_def.signature_def['serving_default'].outputs

# # Just get the first thing(s) from the serving signature def.  i.e. this
# # model only has a single input and a single output.
# input_name = None
# for k,v in inputs.items():
#     input_name = v.name
#     #break

# output_name = None
# for k,v in outputs.items():
#     output_name = v.name
#     #break

# # Make a dictionary that maps Earth Engine outputs and inputs to
# # AI Platform inputs and outputs, respectively.
# import json
# input_dict = "'" + json.dumps({input_name: "array"}) + "'"
# output_dict = "'" + json.dumps({output_name: "output"}) + "'"
# print(input_dict)
# print(output_dict)

In [ ]:
#tfdf.model_plotter.plot_model_in_colab(model_1)#, tree_idx=0, max_depth=3)
model_1.summary()

In [3]:
# Put the EEified model next to the trained model directory.
MODEL_DIR = 'gs://congo_deforestation/model'

from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
print(inputs.items())
for k,v in inputs.items():
    input_name = v.name
    break

output_name = None
for k,v in outputs.items():
    output_name = v.name
    break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: "output"}) + "'"
print(input_dict)
print(output_dict)

EEIFIED_DIR = 'gs://congo_deforestation/ee_model'
PROJECT = 'drivers-deforestation2'
# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

ItemsView({'elevation': name: "serving_default_elevation:0"
dtype: DT_INT64
tensor_shape {
  dim {
    size: -1
  }
}
, 'ruralComplexDistance': name: "serving_default_ruralComplexDistance:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
}
, 'miningImage': name: "serving_default_miningImage:0"
dtype: DT_INT64
tensor_shape {
  dim {
    size: -1
  }
}
, 'navigableRiversDistance': name: "serving_default_navigableRiversDistance:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
}
, 'localitiesDistance': name: "serving_default_localitiesDistance:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
}
, 'protectedAreasImage': name: "serving_default_protectedAreasImage:0"
dtype: DT_INT64
tensor_shape {
  dim {
    size: -1
  }
}
, 'slope': name: "serving_default_slope:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
}
, 'roadsDistance': name: "serving_default_roadsDistance:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
}
, 'concessionsImage': name:

In [7]:
MODEL_NAME = 'congo_model2'
VERSION_NAME = 'v0'
REGION = 'us-central1'


# !gcloud ai-platform models create {MODEL_NAME} \
#   --project {PROJECT} \
#   --region {REGION} \
#   --enable-logging

!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --region {REGION} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --framework "TENSORFLOW" \
  --runtime-version=2.7 \
  --python-version=3.7 \


Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in projects [drivers-deforestation2] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name


In [ ]:
# [END upload-model]